# Hail DB Data Extraction

# `.env`

- Make sure to create a `.env` file with:
```
AZURE_ENDPOINT=...
AZURE_API_KEY=...
AZURE_MODEL=...
```

# Define LLM Inference Code

In [1]:
!pip install openai python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


In [2]:
import os
import re
import json
from openai import AzureOpenAI
from typing import Optional

class GPTModel:
  def __init__(self, api_key: str, endpoint: str, model: str):
    self.client = AzureOpenAI(
      azure_endpoint = endpoint,
      api_key = api_key,
      api_version = "2024-02-01"
    )
    self.model = model

  @staticmethod
  def extract_json_content(s: str) -> str:
    # Define the regex pattern to match `json<content>`
    pattern = r'^```json(.*)```$'

    # Search for the pattern
    match = re.search(pattern, s, re.DOTALL)

    # If a match is found, return the content within the wrapper
    if match:
        return match.group(1).strip()
    else:
        # Return the original string or handle as needed
        return s

  @staticmethod
  def json_to_dict(s: str) -> dict:
    return json.loads(s)

  def call(self, user_msg: str, sys_msg: Optional[str] = None) -> str:
    messages = []

    if sys_msg:
      messages.append({"role": "system", "content": sys_msg})
    messages.append({"role": "user", "content": user_msg})

    response = self.client.chat.completions.create(
        model=self.model,
        messages=messages,
    )

    res = response.choices[0].message.content
    res = GPTModel.extract_json_content(res)
    res = GPTModel.json_to_dict(res)
    return res

In [3]:
from dotenv import load_dotenv
load_dotenv()

AZURE_ENDPOINT = os.getenv('AZURE_ENDPOINT')
AZURE_API_KEY = os.getenv('AZURE_API_KEY')
AZURE_MODEL = os.getenv('AZURE_MODEL')

gpt_model = GPTModel(
    api_key=AZURE_API_KEY,
    endpoint=AZURE_ENDPOINT,
    model=AZURE_MODEL
)

# Load Dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

df_hail_db = pd.read_csv('/content/drive/My Drive/Hackathon_20231007/Hail_db/hail_db_with_size_v20230204.csv')

In [6]:
import pandas as pd

df_hail_reference_objects = pd.read_csv('/content/drive/My Drive/Hackathon_20231007/Hail_db/HailReferenceObjects.csv')

In [7]:
hail_sizes_sorted = [hail_size for hail_size in sorted(df_hail_reference_objects["Reference Object"].tolist()) if hail_size != "unknown"]

In [8]:
print('\n'.join(hail_sizes_sorted))

3/4 of golf ball
M & M
acorn
apple
baseball
blueberry
bottle cap
button
cherry
chickpea
chocolate chip
cochonnet
dime
double pea
estimated 1 inch
estimated 1.5 inch
estimated 100mm
estimated 10mm
estimated 11mm
estimated 12mm
estimated 14mm
estimated 15mm
estimated 18mm
estimated 19mm
estimated 2 inches
estimated 20mm
estimated 21mm
estimated 22mm
estimated 24mm
estimated 25mm
estimated 26mm
estimated 27mm
estimated 28mm
estimated 30mm
estimated 32mm
estimated 35mm
estimated 38mm
estimated 40mm
estimated 45mm
estimated 50mm
estimated 55mm
estimated 60mm
estimated 65mm
estimated 6mm
estimated 70mm
estimated 75mm
estimated 80mm
estimated 90mm
estimated 95mm
estimated half inch
golf ball
grape
grapefruit
half dime
half dollar
half golf ball
half penny
hand palm (F)
hand palm (M)
hen egg
ice cube
large
large grape
large marble
larger than baseball
larger than dime
larger than golf ball
larger than hen egg
larger than loonie
larger than marble
larger than nickel
larger than pea
larger than 

# Extraction Information

In [9]:
!pip install fuzzywuzzy python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 49.5 MB/s eta 0:00:00


In [10]:
from pydantic import BaseModel

class Response(BaseModel):
  reference_object: str
  reference_location: str
  reference_object_flag: str
  picture_flag: str
  source: str
  social_media_link: str
  hail_start_time: str
  time_flag: str
  hail_damage: str
  hail_accumulation: str
  associated_rain_amount: str
  associated_severe_weather: str

response_example = Response(
    reference_object="marble",
    reference_location="Toronto, ON",
    reference_object_flag="visual estimate from up-close",
    picture_flag="picture",
    source="social media",
    social_media_link="https://www.twitter.com",
    hail_start_time="2023-01-01 00:00:00",
    time_flag="watch",
    hail_damage="unknown",
    hail_accumulation="unknown",
    associated_rain_amount="unknown",
    associated_severe_weather="unknown"
)

def extract_info_from_note(gpt_model, df_hail_db, hail_sizes, row_index):
  note = df_hail_db.loc[row_index, 'Notes']

  newline = "\n"

  sys_msg = f"Your response to the user's instruction should follow the following JSON format: {response_example.json()}"

  user_msg = f"""
Instruction:
Given the following note about hail observation below, extract the following information:
- reference_object: size of the observed hail, choose one of the hail size options from below
- reference_location: location of the observed hail, such as neighborhood/intersection, town, and/or province code in capital letter
- reference_object_flag: how the hail size was measured (ex: caliper, ruler, measuring tape, side by side with <reference object>, visual estimate from up-close, visual estimate from far-off, etc...)
- picture_flag: was a picture or video taken (e.g.: picture, video)
- source: where did the note come from (ex: social media, phone-in, phone-out, email, storm spotter, Nav Canada stations at airports, CoCoRaSH, etc...)
- social_media_link: social media link if it exists
- hail_start_time: esimated start time of hail in the format of YYYY-MM-DD HH:MM:SS
- time_flag: how was the time estimated (ex: radar, watch, GPS, approximate, timing of social media post)
- hail_damage: damage/injury associated with hail
- hail_accumulation: how much hail has accumulated (ex: sparse, fully covered, Xmm of accumulation)
- associated_rain_amount: how much rain was associated with the hail
- associated_severe_weather: any other severe weather associated with the hail (ex: tornado, high wind, heavy wind)
For any of the fields above, if the information cannot be found or is false, output: unknown

Hail Size Options:
{newline.join(hail_sizes)}

Note:
{note}
"""

  res = gpt_model.call(user_msg, sys_msg)

  return res

In [11]:
from fuzzywuzzy import process

def find_closest_match(items, target):
    if target in set(items):
      return target

    closest_match, score = process.extractOne(target, items)

    return closest_match

In [19]:
# Adjust as needed
start_index = 5000
end_index = 7000
thread_max_workers = 5

In [20]:
%%time
import pandas as pd
from google.colab import files
from tqdm import tqdm

# Extracting Information

from concurrent.futures import ThreadPoolExecutor, as_completed

def process_row(i, gpt_model, df_hail_db, hail_sizes_sorted):
    try:
        res = extract_info_from_note(
            gpt_model=gpt_model,
            df_hail_db=df_hail_db,
            hail_sizes=hail_sizes_sorted,
            row_index=i,
        )
        return {
            "row_index": i,
            "success": True,
            "data": {
                "row_index": i,
                "reference_object": res.get('reference_object', 'unknown'),
                "reference_location": res.get('reference_location', 'unknown'),
                "reference_object_flag": res.get('reference_object_flag', 'unknown'),
                "picture_flag": res.get('picture_flag', 'unknown'),
                "source": res.get('source', 'unknown'),
                "social_media_link": res.get('social_media_link', 'unknown'),
                "hail_start_time": res.get('hail_start_time', 'unknown'),
                "time_flag": res.get('time_flag', 'unknown'),
                "hail_damage": res.get('hail_damage', 'unknown'),
                "hail_accumulation": res.get('hail_accumulation', 'unknown'),
                "associated_rain_amount": res.get('associated_rain_amount', 'unknown'),
                "associated_severe_weather": res.get('associated_severe_weather', 'unknown'),
            }
        }
    except Exception as e:
        print(f"An error occurred processing row {i}: {e}")
        return {"row_index": i, "success": False}

# Example of how you'd call this in your main code
df_extracted_info = []
missed_index_list = []

with ThreadPoolExecutor(max_workers=thread_max_workers) as executor:
    # Prepare futures
    futures = [executor.submit(process_row, i, gpt_model, df_hail_db, hail_sizes_sorted)
               for i in range(start_index, end_index)]

    # Process results as they become available, with tqdm for progress tracking
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing rows"):
        result = future.result()
        if result["success"]:
            df_extracted_info.append(result["data"])
        else:
            missed_index_list.append(result["row_index"])

print(f"Missed indices: {missed_index_list}")

# Matching Reference Objects

hail_sizes_sorted_with_unknown = hail_sizes_sorted + ['unknown']

for row in df_extracted_info:
  matched_reference_object = find_closest_match(hail_sizes_sorted_with_unknown, row['reference_object'])
  if matched_reference_object != row['reference_object']:
    print(f"Reference object mismatch for row {row['row_index']}: {matched_reference_object} replacing {row['reference_object']}")
  row['reference_object'] = matched_reference_object

# Saving File

df_extracted_info_pd = pd.DataFrame(df_extracted_info)
df_extracted_info_pd = df_extracted_info_pd.sort_values(by='row_index', ascending=True)
print(df_extracted_info_pd)
filename = f'extracted_info_{start_index}_to_{end_index}.csv'
df_extracted_info_pd.to_csv(filename, index=False)
files.download(filename)

Processing rows:  29%|██▊       | 574/2000 [28:48<1:03:40,  2.68s/it]

An error occurred processing row 5564: Extra data: line 2 column 1 (char 401)


Processing rows: 100%|██████████| 2000/2000 [1:29:43<00:00,  2.69s/it]


Missed indices: [5564]
Reference object mismatch for row 5002: nickel replacing nickel to quarter
Reference object mismatch for row 5003: larger than tennis ball replacing tennis ball
Reference object mismatch for row 5006: golf ball replacing quarter to golf ball
Reference object mismatch for row 5012: loonie replacing quarter to loonie
Reference object mismatch for row 5021: nickel replacing pea to nickel
Reference object mismatch for row 5067: nickel replacing nickel to quarter
Reference object mismatch for row 5109: marble replacing marble to quarter
Reference object mismatch for row 5111: nickel replacing pea to nickel
Reference object mismatch for row 5114: penny replacing smaller than penny
Reference object mismatch for row 5154: nickel replacing nickel to quarter
Reference object mismatch for row 5166: dime replacing dime to nickel
Reference object mismatch for row 5184: marble replacing pea to marble
Reference object mismatch for row 5198: nickel replacing pea to nickel sized 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CPU times: user 45.7 s, sys: 3.3 s, total: 49 s
Wall time: 1h 29min 45s


> Note

- It takes around 40min to process 1k rows.

# Combine Dataset

In [33]:
import pandas as pd

df_extracted_info_0_to_1000 = pd.read_csv('extracted_info_0_to_1000.csv')
df_extracted_info_1000_to_3000 = pd.read_csv('extracted_info_1000_to_3000.csv')
df_extracted_info_3000_to_5000 = pd.read_csv('extracted_info_3000_to_5000.csv')
df_extracted_info_5000_to_7000 = pd.read_csv('extracted_info_5000_to_7000.csv')

In [37]:
from google.colab import files

# 1. Combine Multiple DataFrames
df_extracted_info_combined = pd.concat([df_extracted_info_0_to_1000, df_extracted_info_1000_to_3000, df_extracted_info_3000_to_5000, df_extracted_info_5000_to_7000])

# 2. Check for Missing Rows and Insert New Rows
full_range = pd.DataFrame({'row_index': range(7000)})  # Assuming row_index goes from 0 to 6999
df_extracted_info_combined = df_extracted_info_combined.merge(full_range, on='row_index', how='right')

df_extracted_info_combined.fillna('unknown', inplace=True)  # Replace NaN with 'unknown'

# 3. Sort the DataFrame by 'row_index'
df_extracted_info_combined = df_extracted_info_combined.sort_values(by='row_index')

# 4. Save the DataFrame as a CSV file
df_extracted_info_combined.to_csv('extracted_info_combined.csv', index=False)
files.download('extracted_info_combined.csv')